In [1]:
import tensorflow as tf
import os
import time
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout, ReLU, concatenate, ZeroPadding2D
from tensorflow.keras.layers import BatchNormalization, Activation, LeakyReLU, UpSampling2D, Conv2D, Conv2DTranspose
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing import image
from IPython import display
import cv2
 

PATH = 'D:/01_data/'

def load_xy():
    input_image_path = PATH + 'padding_img/'
    target_image_path = PATH + 'resize_img/'
    
    for i in range(30):
        if i==0:
            input_image = cv2.imread(input_image_path+'{}.jpg'.format(str(i)))
            target_image = cv2.imread(target_image_path+'{}.jpg'.format(str(i)))
            input_images = input_image.reshape(1,input_image.shape[0],input_image.shape[1],3)
            target_images = target_image.reshape(1,target_image.shape[0],target_image.shape[1],3)
        else:
            try:
                input_image = cv2.imread(input_image_path+'{}.jpg'.format(str(i)))
                target_image = cv2.imread(target_image_path+'{}.jpg'.format(str(i)))
                input_image = input_image.reshape(1,input_image.shape[0],input_image.shape[1],3)
                target_image = target_image.reshape(1,target_image.shape[0],target_image.shape[1],3)
                input_images = np.concatenate([input_images,input_image])
                target_images = np.concatenate([target_images,target_image])
            except:
                continue
    
    return input_images,target_images

# normalizing the images to [-1, 1]

def normalize(input_image, target_image):
    # 전처리
    for i in range(len(input_image)):
        if i==0:
            input_images = (input_image[i]/ 127.5) - 1
            target_images = (target_image[i]/ 127.5) - 1
            input_images,target_images = input_images.reshape(1,input_images.shape[0],input_images.shape[1],3),target_images.reshape(1,target_images.shape[0],target_images.shape[1],3)
        
        else:
            temp_1 = input_image[i].reshape(1,input_image.shape[1],input_image.shape[2],3)/127.5 - 1
            temp_2 = target_image[i].reshape(1,target_image.shape[1],target_image.shape[2],3)/127.5 - 1
            input_images = np.concatenate([input_images,temp_1])
            target_images = np.concatenate([target_images,temp_2])

    return input_images, target_images

input_images, target_images = load_xy()
print(input_images.shape, target_images.shape)
input_images, target_images = normalize(input_images, target_images)
input_images.shape, target_images.shape

(16, 720, 1280, 3) (16, 720, 1280, 3)


((16, 720, 1280, 3), (16, 720, 1280, 3))

In [17]:

inputs = Input(shape=[720, 1280, 3])
down_stack0 = Conv2D(64, 4, strides=2, padding='same',kernel_initializer=tf.random_normal_initializer(0., 0.02), use_bias=False)(inputs)# (bs, 128, 128, 64)
down_stack1 = Conv2D(128, 4, strides=2, padding='same',kernel_initializer=tf.random_normal_initializer(0., 0.02), use_bias=False)(down_stack0)# (bs, 128, 128, 64)
down_stack2 = BatchNormalization()(down_stack1)
down_stack3 = LeakyReLU()(down_stack2)
down_stack4 = Conv2D(256, 4, strides=2, padding='same',kernel_initializer=tf.random_normal_initializer(0., 0.02), use_bias=False)(down_stack3)
down_stack5 = BatchNormalization()(down_stack4)
down_stack6 = LeakyReLU()(down_stack5)
down_stack7 = Conv2D(512, 4, strides=2, padding='same',kernel_initializer=tf.random_normal_initializer(0., 0.02), use_bias=False)(down_stack6)
down_stack8 = BatchNormalization()(down_stack7)
down_stack9 = LeakyReLU()(down_stack8)
down_stack10 = Conv2D(512, 4, strides=2, padding='same',kernel_initializer=tf.random_normal_initializer(0., 0.02), use_bias=False)(down_stack9)
down_stack11 = BatchNormalization()(down_stack10)
down_stack12 = LeakyReLU()(down_stack11)
down_stack13 = Conv2D(512, 4, strides=2, padding='same',kernel_initializer=tf.random_normal_initializer(0., 0.02), use_bias=False)(down_stack12)
down_stack14 = BatchNormalization()(down_stack13)
down_stack15 = LeakyReLU()(down_stack14)
down_stack16 = Conv2D(512, 4, strides=2, padding='same',kernel_initializer=tf.random_normal_initializer(0., 0.02), use_bias=False)(down_stack15)
down_stack17 = BatchNormalization()(down_stack16)
down_stack18 = LeakyReLU()(down_stack17)
down_stack19 = Conv2D(512, 4, strides=2, padding='same',kernel_initializer=tf.random_normal_initializer(0., 0.02), use_bias=False)(down_stack18)
down_stack20 = BatchNormalization()(down_stack19)
down_stack21 = LeakyReLU()(down_stack20)
down_stack22 = Conv2D(512, 4, strides=2, padding='same',kernel_initializer=tf.random_normal_initializer(0., 0.02), use_bias=False)(down_stack21)
down_stack23 = BatchNormalization()(down_stack22)
down_stack24 = LeakyReLU()(down_stack23)


up_stack1 = Conv2DTranspose(512, 4, strides=2,padding='same',kernel_initializer= tf.random_normal_initializer(0., 0.02),use_bias=False)(down_stack24)
up_stack2 = Dropout(0.5)(up_stack1)
up_stack3 = ReLU()(up_stack2)
merge1 = concatenate([down_stack18, up_stack3])
up_stack4 = Conv2DTranspose(512, 4, strides=2,padding='same',kernel_initializer= tf.random_normal_initializer(0., 0.02),use_bias=False)(merge1)
up_stack5 = Dropout(0.5)(up_stack4)
up_stack6 = ReLU()(up_stack5)
merge2 = concatenate([down_stack15, up_stack6])
up_stack7 = Conv2DTranspose(512, 4, strides=2,padding='same',kernel_initializer= tf.random_normal_initializer(0., 0.02),use_bias=False)(merge2)
up_stack8 = Dropout(0.5)(up_stack7)
up_stack9 = ReLU()(up_stack8)
merge3 = concatenate([down_stack12, up_stack9])
up_stack10 = Conv2DTranspose(512, 4, strides=2,padding='same',kernel_initializer= tf.random_normal_initializer(0., 0.02),use_bias=False)(merge3)
merge4 = concatenate([down_stack9, up_stack10])
up_stack11 = Conv2DTranspose(256, 4, strides=2,padding='same',kernel_initializer= tf.random_normal_initializer(0., 0.02),use_bias=False)(merge4)
merge5 = concatenate([down_stack6, up_stack11]) 
up_stack12 = Conv2DTranspose(128, 4, strides=2,padding='same',kernel_initializer= tf.random_normal_initializer(0., 0.02),use_bias=False)(merge5)
merge6 = concatenate([down_stack3, up_stack12])
up_stack13 = Conv2DTranspose(64, 4, strides=2,padding='same',kernel_initializer= tf.random_normal_initializer(0., 0.02),use_bias=False)(merge6)
merge7 = concatenate([down_stack0, up_stack13])
g_out_put = Conv2DTranspose(3, 4,strides=2,padding='same',kernel_initializer=tf.random_normal_initializer(0., 0.02),activation='tanh')(merge7) # (bs, 256, 256, 3)

generator = Model(inputs = inputs, outputs = g_out_put)
generator.summary()

ValueError: A `Concatenate` layer requires inputs with matching shapes except for the concat axis. Got inputs shapes: [(None, 6, 10, 512), (None, 4, 6, 512)]

In [18]:

inp = Input(shape=[720, 1280, 3], name='input_image')
tar = Input(shape=[720, 1280, 3], name='target_image')

merge1 = concatenate([inp, tar])# (bs, 256, 256, channels*2)
discriminator1 = Conv2D(64, 4, strides=2, padding='same',kernel_initializer=tf.random_normal_initializer(0., 0.02), use_bias=False)(merge1)
discriminator2 = Conv2D(128, 4, strides=2, padding='same',kernel_initializer=tf.random_normal_initializer(0., 0.02), use_bias=False)(discriminator1)
discriminator3 = BatchNormalization()(discriminator2)
discriminator4 = LeakyReLU()(discriminator3)
discriminator5 = Conv2D(256, 4, strides=2, padding='same',kernel_initializer=tf.random_normal_initializer(0., 0.02), use_bias=False)(discriminator4)
discriminator6 = BatchNormalization()(discriminator5)
discriminator7 = LeakyReLU()(discriminator6)
discriminator8 = ZeroPadding2D()(discriminator7)
discriminator9 = Conv2D(512, 4, strides=1, padding='same',kernel_initializer=tf.random_normal_initializer(0., 0.02), use_bias=False)(discriminator8)
discriminator10 = BatchNormalization()(discriminator9)
discriminator11 = LeakyReLU()(discriminator10)
discriminator12 = ZeroPadding2D()(discriminator11)
d_out_put = Conv2D(1, 4, strides=1,kernel_initializer=tf.random_normal_initializer(0., 0.02))(discriminator12)

discriminator = Model(inputs = [inp, tar], outputs = d_out_put)

discriminator.compile(loss = 'binary_crossentropy', optimizer=Adam(learning_rate=0.0002,beta_1=0.5))
discriminator.trainable = False
discriminator.summary()

dis_output = discriminator([generator([input_images, target_images]),target_images])
gen_in = generator([input_images, target_images])
gan = Model(gen_in, dis_output)
gan.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.0002,beta_1=0.5))
gan.summary()

accuracies = []
losses = []

Model: "functional_11"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_image (InputLayer)        [(None, 720, 1280, 3 0                                            
__________________________________________________________________________________________________
target_image (InputLayer)       [(None, 720, 1280, 3 0                                            
__________________________________________________________________________________________________
concatenate_37 (Concatenate)    (None, 720, 1280, 6) 0           input_image[0][0]                
                                                                 target_image[0][0]               
__________________________________________________________________________________________________
conv2d_131 (Conv2D)             (None, 360, 640, 64) 6144        concatenate_37[0][0] 

UnknownError: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above. [Op:Conv2D]